Import Libraries

In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import csv 
import re
import pandas as pd 
from skimage.feature import greycomatrix, greycoprops

In [1]:
#this one works
img = cv2.imread('dataset/test/MildDemented/3_0.jpg')

if img is None:
    print('Wrong path:', path)
else:
    img = cv2.resize(img, dsize=(100,200))
    cv2.imwrite('dataset/test/MildDemented/3_0.jpg',img)


Image Preprocessing

In [25]:
def resize_images(dataset_dir):
    for folder in os.listdir(dataset_dir):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder))
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, filename))
            if img is None:
                print('Wrong path:', (os.path.join(dataset_dir, folder, filename)))
            else:
                img = cv2.resize(img, dsize=(100,200))
                cv2.imwrite(os.path.join(folder, filename),img)
                print(i)

In [27]:
images = resize_images("dataset/test")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
11

Histogram Equalization

In [2]:
#this works

# import Opencv
import cv2
  
# import Numpy
import numpy as np
  
# read a image using imread
img = cv2.imread('dataset/test/MildDemented/3_0.jpg', 0)
  
# creating a Histograms Equalization
# of a image using cv2.equalizeHist()
equ = cv2.equalizeHist(img)
  
# stacking images side-by-side
res = np.hstack((img, equ))
  
# show image input vs output
cv2.imshow('image', img)
  
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
def save_histogram_equization_images_to_folder(dataset_dir):
    for folder in os.listdir(dataset_dir):
        for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
            sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
            len_sub_folder = len(sub_folder_files) - 1
            for i, filename in enumerate(sub_folder_files):
                img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            if img is not None:  
                equ = cv2.equalizeHist(img)
                cv2.imwrite(os.path.join(folder, filename),equ)

In [16]:
save_histogram_equization_images_to_folder("dataset")

NotADirectoryError: [WinError 267] The directory name is invalid: 'dataset/train\\MildDemented\\3_0.jpg'

WCE

In [ ]:
# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.split("_")
    return str_[0]

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")

In [ ]:
dataset_dir = "dataset/" 

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            gray = cv2.cvtColor(img, 0)
            imgs.append(gray)
            labels.append(normalize_label(filename))
            descs.append(normalize_desc(folder, sub_folder))

In [ ]:
# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, distances=dists, angles=agls, levels=lvl, symmetric=sym, normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature

In [ ]:
# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for i in range(4):
    for img, label in zip(imgs[i], labels): 
        glcm_all_agls.append(calc_glcm_all_agls(img, label, props=properties))
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

In [ ]:
# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

#save to csv
glcm_df.to_csv("glcm_dataset.csv")

glcm_df.head(7)

PNN

In [ ]:
def input():
    x_train = pd.read_csv("glcm_dataset_train.csv", usecols = ['dissimilarity_0','dissimilarity_45','dissimilarity_90','dissimilarity_135',
                                                  'correlation_0','correlation_45','correlation_90','correlation_135',
                                                   'homogeneity_0','homogeneity_45','homogeneity_90','homogeneity_135',
                                                  'contrast_0','contrast_45','contrast_90','contrast_135',
                                                   'ASM_0','ASM_45','ASM_90','ASM_135',
                                                   'energy_0','energy_45','energy_90','energy_135'])
    y_train = pd.read_csv("glcm_dataset_train.csv", usecols = ['label'])
    x_test = pd.read_csv("glcm_dataset_test.csv", usecols = ['dissimilarity_0','dissimilarity_45','dissimilarity_90','dissimilarity_135',
                                                  'correlation_0','correlation_45','correlation_90','correlation_135',
                                                   'homogeneity_0','homogeneity_45','homogeneity_90','homogeneity_135',
                                                  'contrast_0','contrast_45','contrast_90','contrast_135',
                                                   'ASM_0','ASM_45','ASM_90','ASM_135',
                                                   'energy_0','energy_45','energy_90','energy_135'])
    y_test = pd.read_csv("glcm_dataset_test.csv", usecols = ['label'])
    x = scale(x)

    data = {'x_train': x_train, 
            'x_test': x_test, 
            'y_train': y_train, 
            'y_test': y_test}

    return data

# Helper function that combines the pattern layer and summation layer
def rbf(centre, x, sigma):
    centre = centre.reshape(1, -1)

    temp = -np.sum((centre - x) ** 2, axis = 1)
    temp = temp / (2 * sigma * sigma)
    temp = np.exp(temp)
    gaussian = np.sum(temp)

    return gaussian


def subset_by_class(data, labels):

    x_train_subsets = []

    for l in labels:
        indices = np.where(data['y_train'] == l)
        x_train_subsets.append(data['x_train'][indices, :])

    return x_train_subsets


def PNN(data):

    num_testset = data['x_test'].shape[0]
    labels = np.unique(data['y_train'])
    num_class = len(labels)

    sigma = 10

    # Splits the training set into subsets where each subset contains data points from a particular class	
    x_train_subsets = subset_by_class(data, labels)	

    # Variable for storing the summation layer values from each class
    summation_layer = np.zeros(num_class)

    # Variable for storing the predictions for each test data point
    predictions = np.zeros(num_testset)

    for i, test_point in enumerate(data['x_test']):

        for j, subset in enumerate(x_train_subsets):
            # Calculate summation layer
            summation_layer[j] = np.sum(
                rbf(test_point, subset[0], sigma)) / subset[0].shape[0] 

        # The index having the largest value in the summation_layer is stored as the prediction
        predictions[i] = np.argmax(summation_layer)

    return predictions


def print_metrics(y_test, predictions):

    print('Confusion Matrix')
    print(confusion_matrix(y_test, predictions))
    print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
    print('Precision: {}'.format(precision_score(y_test, predictions, average = 'micro')))
    print('Recall: {}'.format(recall_score(y_test, predictions, average = 'micro')))

In [ ]:
data = input()
predictions = PNN(data)
print_metrics(data['y_test'], predictions)

LR